In [35]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lit
import locale
import sys
import os
import re

In [36]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [37]:
spark

# Leyendo lista de Colegios y Estudiantes

In [38]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [39]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [40]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [41]:
print("Archivos de estudiantes: ", estuDic['20131'],estuDic['20132'])
print("Archivos de colegios: ",coleDic['2013'])

Archivos de estudiantes:  SB11_20131.txt SB11_20132.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2013.txt


In [42]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20131'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')
df2 = spark.read.load(base2 + '/' + estuDic['20132'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [43]:
df1.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_VECESPRESENTOEXAMEN',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENETELEVISOR',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TIENE_NEVERA',
 'FAMI_TIENE_CELULAR',
 'FAMI_TELEFONO',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_RECIBESALARIO',
 'ESTU_ANTECEDENTES',
 'ESTU_ANOSPREESCOLAR',
 'ESTU_ANOMATRICULAPRIMERO',
 'ESTU_ANOTERMINOQUINTO',
 'ESTU_ANOMATRICULASEXTO',
 'ES

In [44]:
df2.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_VECESPRESENTOEXAMEN',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENETELEVISOR',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TIENE_NEVERA',
 'FAMI_TIENE_CELULAR',
 'FAMI_TELEFONO',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_RECIBESALARIO',
 'ESTU_ANTECEDENTES',
 'ESTU_ANOSPREESCOLAR',
 'ESTU_ANOMATRICULAPRIMERO',
 'ESTU_ANOTERMINOQUINTO',
 'ESTU_ANOMATRICULASEXTO',
 'ES

In [45]:
columnasPuntos20131 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_MATEMATICAS',
 'PUNT_INGLES',
 'PUNT_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_FISICA',
 'PUNT_INTERDISC_VIOLENCIAYSOC',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_PROFUNDIZA_MATEMATICA']

columnasPuntos20132 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_MATEMATICAS',
 'PUNT_PROFUNDIZA_MATEMATICA',
 'PUNT_CIENCIAS_SOCIALES',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_INGLES',
 'PUNT_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_FISICA',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_INTERDISC_VIOLENCIAYSOC'  
]

In [47]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20131'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos20132)
df2 = spark.read.load(base2 + '/' + estuDic['20132'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos20132)

In [48]:

cols = columnasPuntos20132
cols.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols.remove('COLE_NOMBRE_ESTABLECIMIENTO')

In [49]:
# SE unen los dos dataframes
df = unionAll(df1,df2)
for cp in cols:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    df = df.na.fill({cp:0})
df = df.na.fill({'COLE_COD_DANE_ESTABLECIMIENTO':'--'})
df = df.na.fill({'COLE_NOMBRE_ESTABLECIMIENTO':'--'})

In [50]:
# SE evalua sí tiene valores nullos
print("Nùmero de filas con cole_cod_dane_institucion nulos: ", df.filter("COLE_COD_DANE_ESTABLECIMIENTO = '--'").count())
print("Nùmero de filas con cole_nombre_sede nulos: ", df.filter("COLE_NOMBRE_ESTABLECIMIENTO = '--'").count())

Nùmero de filas con cole_cod_dane_institucion nulos:  0
Nùmero de filas con cole_nombre_sede nulos:  0


In [51]:
df = df.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [52]:
print("Numero de filas ", df.count())

Numero de filas  576092


In [53]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)

In [56]:
dsCountEst = dsCountEst.withColumnRenamed("count","num_est")
dsCountEst.show(10,False)
# Numero de estudiantes por instituciòn

+-----------------------------+-------+
|COLE_COD_DANE_ESTABLECIMIENTO|num_est|
+-----------------------------+-------+
|311001105863                 |1285   |
|311001105391                 |1144   |
|105001000108                 |1103   |
|105001013340                 |725    |
|105615000236                 |708    |
|305001003963                 |661    |
|311001104786                 |658    |
|176001005813                 |618    |
|105154000301                 |607    |
|168001003591                 |551    |
+-----------------------------+-------+
only showing top 10 rows



In [57]:
rowmax = dsCountEst.agg({"num_est": "max"}).collect()[0]
rowmin = dsCountEst.agg({"num_est": "min"}).collect()[0]

num_est_max = rowmax["max(num_est)"]
num_est_min = rowmin["min(num_est)"]

In [58]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['PUNT_MATEMATICAS']+df['PUNT_INGLES']+df['PUNT_CIENCIAS_SOCIALES']+
                     df['PUNT_BIOLOGIA']+df['PUNT_FILOSOFIA']+df['PUNT_FISICA']+df['PUNT_QUIMICA']+
                     df['PUNT_LENGUAJE']+df['PUNT_INTERDISC_MEDIOAMBIENTE']+df['PUNT_INTERDISC_VIOLENCIAYSOC']+
                     df['PUNT_PROFUNDIZA_BIOLOGIA']+
                     df['PUNT_PROFUNDIZA_CSOCIALES']+df['PUNT_PROFUNDIZA_LENGUAJE']+df['PUNT_PROFUNDIZA_MATEMATICA'])


## Resultados finales por instituciòn

In [59]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df = df.groupBy('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO').mean()

rowprommax = df.agg({"avg(Suma)": "max"}).collect()[0]
rowprommin = df.agg({"avg(Suma)": "min"}).collect()[0]

prom_est_max = rowprommax["max(avg(Suma))"]
prom_est_min = rowprommin["min(avg(Suma))"]

print("Màxima calificaciòn en colegios: ", prom_est_max)
print("Mìxima calificaciòn en colegios: ", prom_est_min)

Màxima calificaciòn en colegios:  601.4285714285714
Mìxima calificaciòn en colegios:  281.25


In [60]:
dfjoin = df.join(dsCountEst,"COLE_COD_DANE_ESTABLECIMIENTO")
dfjoin= dfjoin.withColumn("Indice_Prom",round(dfjoin['avg(Suma)']/prom_est_max,3))
dfjoin= dfjoin.withColumn("Indice_Num",round(dfjoin['num_est']/num_est_max,3))
dfjoin= dfjoin.withColumn("Indice_Total",round(dfjoin['Indice_Prom']*dfjoin['Indice_Num'],3))

In [61]:
dfjoin = dfjoin.select('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','num_est','Indice_Prom','Indice_Num','Indice_Total')
dfjoin = dfjoin.withColumn("rankProm", dense_rank().over(Window.orderBy(desc("Indice_Prom"))))
dfjoin = dfjoin.withColumn("rankNum", dense_rank().over(Window.orderBy(desc("Indice_Num"))))
dfjoin = dfjoin.withColumn("rankTotal", dense_rank().over(Window.orderBy(desc("Indice_Total"))))
dfjoin = dfjoin.withColumn("Media",round(dfjoin['avg(Suma)'],2))
dfjoin.printSchema()

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = false)
 |-- avg(Suma): double (nullable = true)
 |-- num_est: long (nullable = false)
 |-- Indice_Prom: double (nullable = true)
 |-- Indice_Num: double (nullable = true)
 |-- Indice_Total: double (nullable = true)
 |-- rankProm: integer (nullable = true)
 |-- rankNum: integer (nullable = true)
 |-- rankTotal: integer (nullable = true)
 |-- Media: double (nullable = true)



In [62]:
# Se ordena por Indice_Prom
dfjoin = dfjoin.orderBy('rankProm',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Prom','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+-----------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Prom|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+-----------+-------+--------+-------+---------+
|       COLEGIO BILINGUE ...|601.43|        1.0|     14|       1|    291|      196|
|       INST ALBERTO MERA...|597.06|      0.993|     34|       2|    276|      181|
|       GIMN VERMONT     ...|586.61|      0.975|     94|       3|    229|      136|
|        COL NUEVO CAMBRIDGE| 583.8|      0.971|     44|       4|    268|      174|
|       COLEGIO LEONARDO ...| 584.0|      0.971|      1|       4|    301|      206|
|       COL LA QUINTA DEL...|580.76|      0.966|     25|       5|    283|      189|
|       COLEGIO SEMINARIO...|580.32|      0.965|     28|       6|    280|      186|
|       COL SAN JORGE DE ...|579.05|      0.963|     55|       7|    259|      166|
|             COL MONTESSORI| 575.6|      0.957|     63|       8|    253|   

In [63]:
# Se ordena por Indice_Num
dfjoin = dfjoin.orderBy('rankNum',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Num','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+----------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Num|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+----------+-------+--------+-------+---------+
|       CENT JOHANN KEPLE...|362.98|       1.0|   1285|     334|      1|        1|
|       COLEGIO TECNISIST...|362.68|      0.89|   1144|     335|      2|        3|
|             INST EDUC CEFA|411.76|     0.858|   1103|     253|      3|        2|
|       INST EDUC INEM JO...|430.53|     0.564|    725|     222|      4|        4|
|       I. E. ESCUELA NOR...|368.99|     0.551|    708|     324|      5|        6|
|       LIC SALAZAR Y HER...|446.58|     0.514|    661|     195|      6|        5|
|       COL BRITANICO SED...|368.71|     0.512|    658|     325|      7|        8|
|       INSTITUCION EDUCA...|390.97|     0.481|    618|     288|      8|        9|
|       I. E. LICEO CAUCASIA|356.98|     0.472|    607|     344|      9|       11|
|   

In [64]:
# Se ordena por Indice_Total
dfjoin = dfjoin.orderBy('rankTotal',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Total','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+------------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Total|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+------------+-------+--------+-------+---------+
|       CENT JOHANN KEPLE...|362.98|       0.604|   1285|     334|      1|        1|
|             INST EDUC CEFA|411.76|       0.588|   1103|     253|      3|        2|
|       COLEGIO TECNISIST...|362.68|       0.537|   1144|     335|      2|        3|
|       INST EDUC INEM JO...|430.53|       0.404|    725|     222|      4|        4|
|       LIC SALAZAR Y HER...|446.58|       0.382|    661|     195|      6|        5|
|       I. E. ESCUELA NOR...|368.99|       0.338|    708|     324|      5|        6|
|       COLEGIO DE LA UNI...|477.06|       0.337|    546|     145|     11|        7|
|       COL BRITANICO SED...|368.71|       0.314|    658|     325|      7|        8|
|       INSTITUCION EDUCA...|390.97|       0.313|    618|     288

## Colegios

In [28]:
# SE crea Dataframe de colegios en este año desaparecio la columna COLE_CODMPIO_COLEGIO
dfc = spark.read.load(base + '/' + coleDic['2013'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))

In [29]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- CLPL_N: integer (nullab

In [30]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [31]:
dfc = dfc.select(colsColegio)

In [32]:
dfc.show(5,False)

+-------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|120717             |7                     |7           |7          |6            |7             |6               |7            |6          |null          |null         |
|6833               |7                     |6           |7          |5            |7             |7               |6            |7          |null          |null         |
|6817               |8                     |7           |7          |7            |7             |7               |8            |7          |null

In [33]:
print("El numero de colegios es: ", dfc.select('COLE_CODIGO_COLEGIO').count())

El numero de colegios es:  8139


## Anàlisis comparativo entre estudiantes y colegios

In [34]:
dfjoin = df1.join(dfc, df1.COLE_COD_DANE_ESTABLECIMIENTO == dfc.COLE_CODIGO_COLEGIO)

In [35]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe còdigo dane en el archivo de colegios.